# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [3]:
# Your code here
# loading the libraries
import pandas as pd
import sqlite3
conn= sqlite3.connect('Lab.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [4]:
q = ("""
      SELECT firstName, lastname
      FROM employees
      JOIN offices
      USING(officeCode)
      WHERE city = 'Boston'
      """)

pd.read_sql(q, conn)

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [5]:

query2 = ("""
SELECT OF.officeCode, OF.city, COUNT(EMP.employeeNumber) AS number_of_employees
FROM offices AS OF
LEFT JOIN employees AS EMP
USING(officeCode)
GROUP BY OF.officeCode, OF.city
HAVING COUNT(EMP.employeeNumber) = 0;
""")

df = pd.read_sql(query2, conn)
print(df)

   officeCode    city  number_of_employees
0          27  Boston                    0


### How many customers are there per office?

In [37]:
query14 = """SELECT customerNumber, officeCode
FROM customers AS C
JOIN employees AS E
ON C.salesRepEmployeeNumber = E.employeeNumber
JOIN offices AS O
USING (officeCode);
"""
pd.read_sql(query14, conn)


,customerNumber,officeCode
0,103,4
1,112,1
2,114,6
3,119,4
4,121,7
...,...,...
95,486,3
96,487,1
97,489,7
98,495,2


### How many orders did each customer make per product


In [5]:
#How many orders did each customer make per product

query3 = """SELECT C.customerNumber, C.customerName, P.productCode, P.productName, COUNT(OD.orderNumber) AS number_of_orders
FROM customers AS C
JOIN orders AS O USING(customerNumber)
JOIN orderdetails AS OD USING(orderNumber)
JOIN products AS P USING(productCode)
GROUP BY C.customerNumber, C.customerName, P.productCode, P.productName
ORDER BY C.customerNumber, number_of_orders DESC;
"""

pd.read_sql(query3, conn)

,customerNumber,customerName,productCode,productName,number_of_orders
0,103,Atelier graphique,S10_2016,1996 Moto Guzzi 1100i,1
1,103,Atelier graphique,S18_1589,1965 Aston Martin DB5,1
2,103,Atelier graphique,S18_2625,1936 Harley Davidson El Knucklehead,1
3,103,Atelier graphique,S18_2870,1999 Indy 500 Monte Carlo SS,1
4,103,Atelier graphique,S18_3685,1948 Porsche Type 356 Roadster,1
...,...,...,...,...,...
2527,496,Kelly's Gift Shop,S700_2834,ATA: B757-300,1
2528,496,Kelly's Gift Shop,S700_3167,F/A 18 Hornet 1/72,1
2529,496,Kelly's Gift Shop,S700_3505,The Titanic,1
2530,496,Kelly's Gift Shop,S700_3962,The Queen Mary,1


### Which employee has processed the most orders, and how many orders have they processed?

In [13]:
query4 = """SELECT firstName, lastName, SUM(quantityOrdered) AS total_ordered
FROM employees
JOIN customers
ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders
USING (customerNumber)
JOIN orderdetails
USING (orderNumber)
GROUP BY firstName, lastName
ORDER BY total_ordered DESC
LIMIT 1;
"""
pd.read_sql(query4, conn)


,firstName,lastName,total_ordered
0,Gerard,Hernandez,14231


### Which customer has spent the most money on orders and what is their total spend?

In [27]:
query5 = ("""
         SELECT CUS.customerName, CUS.customerNumber, SUM(PAY.amount) AS total_customer_payment
         FROM customers AS CUS
         JOIN payments AS PAY
         USING(customerNumber)
         GROUP BY CUS.customerName, CUS.customerNumber
         ORDER BY total_customer_payment DESC
         """)

pd.read_sql(query5, conn)

,customerName,customerNumber,total_customer_payment
0,Euro+ Shopping Channel,141,715738.98
1,Mini Gifts Distributors Ltd.,124,584188.24
2,"Australian Collectors, Co.",114,180585.07
3,Muscle Machine Inc,151,177913.95
4,"Dragon Souveniers, Ltd.",148,156251.03
...,...,...,...
93,Royale Belge,381,29217.18
94,Frau da Collezione,473,25358.32
95,Atelier graphique,103,22314.36
96,Auto-Moto Classics Inc.,198,21554.26


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [33]:
# Your code here
query6 = """SELECT firstName, lastName, productName
FROM employees
JOIN customers
ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders
USING (customerNumber)
JOIN orderdetails
USING (orderNumber)
JOIN products
USING (productCode)"""
pd.read_sql(query6, conn)

,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang
...,...,...,...
2991,Martin,Gerard,1954 Greyhound Scenicruiser
2992,Martin,Gerard,1950's Chicago Surface Lines Streetcar
2993,Martin,Gerard,Diamond T620 Semi-Skirted Tanker
2994,Martin,Gerard,1911 Ford Town Car


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [17]:
# Your code here
query7 = """SELECT firstName, lastName, SUM(quantityOrdered) AS total_ordered
FROM employees
JOIN customers
ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders
USING (customerNumber)
JOIN orderdetails
USING (orderNumber)
GROUP BY firstName, lastName
ORDER BY employees.lastName;
"""
pd.read_sql(query7, conn)

,firstName,lastName,total_ordered
0,Loui,Bondur,6186
1,Larry,Bott,8205
2,Pamela,Castillo,9290
3,Julie,Firrelli,4227
4,Andy,Fixter,6246
5,Martin,Gerard,4180
6,Gerard,Hernandez,14231
7,Leslie,Jennings,11854
8,Barry,Jones,7486
9,Peter,Marsh,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [26]:
query9 = """SELECT firstName, lastName, SUM(quantityOrdered) AS total_ordered
FROM employees
LEFT JOIN customers
ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders
USING (customerNumber)
JOIN orderdetails
USING (orderNumber)
GROUP BY firstName, lastName
HAVING total_ordered >200
ORDER BY total_ordered DESC;
"""
pd.read_sql(query9, conn)

,firstName,lastName,total_ordered
0,Gerard,Hernandez,14231
1,Leslie,Jennings,11854
2,Pamela,Castillo,9290
3,Larry,Bott,8205
4,Barry,Jones,7486
5,George,Vanauf,7423
6,Peter,Marsh,6632
7,Andy,Fixter,6246
8,Loui,Bondur,6186
9,Steve,Patterson,5561


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!